**You should definetely watch this movie!**

In [1]:
pip install nltk

  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
Using cached nltk-3.9.1-py3-none-any.whl (1.5 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
pip install -q sentence_transformers

In [ ]:
pip install faiss-cpu

In [ ]:
pip install yake

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import transformers
import nltk
import faiss #For calculating similarity between embeddings. 
from sentence_transformers import SentenceTransformer #For Sentence Embeddings.

In [3]:
plots = pd.read_csv('wiki_movie_plots_deduped.csv')

In [47]:
# plots.info() 
subset_500 = plots.sample(n=2000, random_state=42)
subset_500.info()


<class 'pandas.core.frame.DataFrame'>
Index: 2000 entries, 5337 to 8135
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Release Year      2000 non-null   int64 
 1   Title             2000 non-null   object
 2   Origin/Ethnicity  2000 non-null   object
 3   Director          2000 non-null   object
 4   Cast              1917 non-null   object
 5   Genre             2000 non-null   object
 6   Wiki Page         2000 non-null   object
 7   Plot              2000 non-null   object
dtypes: int64(1), object(7)
memory usage: 140.6+ KB


In [8]:
subset_500.head()


,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
5337,1951,The Day the Earth Stood Still,American,Robert Wise,"Michael Rennie, Patricia Neal",science fiction,https://en.wikipedia.org/wiki/The_Day_the_Eart...,"When a flying saucer lands in Washington, D.C...."
9809,1981,The Burning,American,Tony Maylam,"Brian Matthews, Holly Hunter, Jason Alexander",horror,https://en.wikipedia.org/wiki/The_Burning_(film),"One night at Camp Blackfoot, several campers p..."
24075,2012,Nobel Chor,Bengali,Suman Ghosh,"Mithun Chakraborty, Saswata Chatterjee, Sudipt...",suspense / drama,https://en.wikipedia.org/wiki/Nobel_Chor,"The first Asian Nobel Laureate, Rabindranath T..."
19057,1952,Trent's Last Case,British,Herbert Wilcox,"Michael Wilding, Margaret Lockwood, Orson Welles",detective,https://en.wikipedia.org/wiki/Trent%27s_Last_C...,A major international financier is found dead ...
24991,1977,Aafat,Bollywood,Atma Ram,"Navin Nischol, Leena Chandavarkar, Amjad Khan,...",unknown,https://en.wikipedia.org/wiki/Aafat,Inspector Amar and Inspector Chhaya are after ...


In [48]:
subset_500['Genre'].sample(5) 

23976                               romance
17202    action, adventure, fantasy, sci-fi
23187                               unknown
517                                   drama
4877                                  drama
Name: Genre, dtype: object

In [49]:
len(subset_500['Genre'].unique())


343

In [82]:
def get_len(row: str):
    length_all = 0 
    number_of_samples = len(subset_500)
    for delta in plots[row]:
        length_all += len(delta) #We add length of the each plot to 'length_all'.
    
    return round((length_all / number_of_samples))

#This function above will help us to demonstrate average length of the plots

In [83]:
lengths = get_len('Plot') #Average length of the plots.
print(lengths)

37765


In [52]:
sen_transformer = SentenceTransformer('all-mpnet-base-v2') #Sentence Transformer

In [53]:
import yake #Library for keyword extraction

This function below takes movie plots as an input and returns a list which contains all keywords for each movie.

In [74]:
def get_keywords():
    kw_extraction = yake.KeywordExtractor(lan = 'en', n = 3, top = 25)
    
    from tqdm import tqdm
    sentences = []
    total_iteration = len(plots.sample(25000))
    with tqdm(total= total_iteration) as pbar:
        for delta in plots['Plot']:
            texts = []
            keywords = kw_extraction.extract_keywords(delta)
            for keyword, _ in keywords:
                texts.append(keyword.lower())

            joined_text = ','.join(texts)
            sentences.append(joined_text)

            pbar.update(1)
    
    return sentences

In [84]:
from nltk.tokenize import sent_tokenize
from tqdm import tqdm  # Import the progress bar library
import numpy as np  # Ensure this is imported if not already


def get_embeddings(dataframe):
    num_of_sentence = []
    sentences = []
    embeddings_for_each_plot = []
    final_embeddings = []
    prev_delta = 0
    pooling = 'sum'

    # Initialize progress bar for processing plots
    for delta in tqdm(dataframe['Plot'], desc="Processing plots"):
        text = sent_tokenize(delta)
        num_of_sentence.append(len(text))
        sentences.extend(text)

    # Compute embeddings for all sentences
    print("We are here")
    embeddings = sen_transformer.encode(sentences)
    print("Here")
    # Initialize progress bar for embedding aggregation
    for sentence_count in tqdm(num_of_sentence, desc="Aggregating embeddings"):
        plot_embeddings = embeddings[prev_delta:prev_delta + sentence_count]
        embeddings_for_each_plot.append(plot_embeddings)
        prev_delta += sentence_count

    # Aggregate embeddings for each plot
    for sentence_embeddings in tqdm(embeddings_for_each_plot, desc="Finalizing embeddings"):
        if pooling == 'sum':
            aggregated = np.sum(sentence_embeddings, axis=0)
        elif pooling == 'mean':
            aggregated = np.mean(sentence_embeddings, axis=0)
        else:
            raise ValueError("Invalid pooling method. Choose 'sum' or 'mean'.")
        final_embeddings.append(aggregated)

    return final_embeddings


In [85]:
# keywordsALL = get_keywords() #Get all the keywords.

In [86]:
# embeddings = get_embbedings(subset_500)

In [87]:
embeddings = get_embeddings(subset_500)

Processing plots: 100%|██████████| 2000/2000 [00:00<00:00, 2862.40it/s]


We are here
Here


Finalizing embeddings: 100%|██████████| 2000/2000 [00:00<00:00, 68098.19it/s]


In [ ]:
import pickle

# Save to pickle
with open('embeddings.pkl', 'wb') as file:
    pickle.dump(embeddings, file)

In [ ]:
# embeddings = sen_transformer.encode(keywordsALL) #Get the embeddings from keywords.

In [ ]:
#This function takes plot_index, embeddings, number of similar plots that you want to see and 
# returns the index of similar movies in the dataset.

def get_similarity(plot_index, embeddings, num_of_similars):
    embeddings = np.array(embeddings, dtype='float32')

    # It uses L2 norm for calculating similarity
    similarity = faiss.IndexFlatL2(embeddings.shape[1])
    similarity.add(embeddings)

    query_vector = embeddings[plot_index].reshape(1, -1)  
    
    D, I = similarity.search(query_vector, num_of_similars) 

    return I,D

In [ ]:
"""
Takes user input and gives moves similar to user discription.
"""

def get_similarity_from_plot(user_input, embeddings, num_of_similars, print_plot):
    
    embeddings = np.array(embeddings, dtype='float32')

    similarity = faiss.IndexFlatL2(embeddings.shape[1])
    similarity.add(embeddings)
    
    input_embedding = sen_transformer.encode(user_input)
    query_vector = input_embedding.reshape(1, -1)

    D, I = similarity.search(query_vector, num_of_similars)
    
    # Reset the index of the movies DataFrame
    movies_reset_index = subset_500.reset_index(drop=True)

    movie_titles = movies_reset_index.loc[I.reshape(-1), 'Title'].tolist()
    movie_plots = movies_reset_index.loc[I.reshape(-1), 'Plot'].tolist()
    
    print(f'I recommend, {movie_titles}')
    
    if print_plot:
        rand_index = np.random.randint(num_of_similars)
        print(f'Here is the plot for {movie_titles[rand_index]}:\n{movie_plots[rand_index]}')


In [ ]:
indexes,Dick = get_similarity(
    plot_index=1, embeddings=embeddings, num_of_similars=5)
print(indexes.reshape(-1))
print(Dick)

[  1  43 245  20 179]


In [ ]:
text = """
Luke Skywalker stays with his aunt and uncle on a farm on Tatooine. 
He is desperate to get off this planet and get to the Academy like his friends, 
but his uncle needs him for the next harvest. Meanwhile, an evil emperor has taken over the galaxy, 
and has constructed a formidable "Death Star" capable of destroying whole planets. 
Princess Leia, a leader in the resistance movement, acquires plans of the Death Star, 
places them in R2-D2, a droid, and sends him off to find Obi-wan Kenobi. Before he finds him, 
R2-D2 ends up on the Skywalkers' farm with his friend C-3PO. R2-D2 then wanders into the desert, 
and when Luke follows, they eventually come across Obi-wan. Will Luke, Obi-wan, and the two droids 
be able to destroy the Death Star, or will the Emperor rule forever?
"""

It is time to try our 'dummy model'. I copied the plot for Star Wars: Episode IV - A New Hope, let's see if the model can recommend us the similar movies based on its plot.

In [ ]:
get_similarity_from_plot(user_input = text, embeddings = embeddings, num_of_similars = 4, print_plot = False)

#If print_plot set True you can print the plots for movies that our model recommend.

I recommend, ['Deaf Sam-yong', 'Wu Kong', 'Jakkanna', "Rustlers' Hideout"]


Wow, It knows the plot is about 'wars' in the stars and hey look, it recommend another movie except Star Wars franchise! Let's try again with different plot!

In [ ]:
text2 = """ In sun-kissed Los Angeles, a pair of morning commuters jammed on a wide stretch of highway are off to a rocky start. But suave Sebastian, a charming jazz pianist whom life's got him on the ropes, and lovely Mia, a brilliant playwright waiting for her big break, are meant for each other. On the one hand, Sebastian thinks he has a serious plan for the future. On the other hand, Mia squanders herself in meaningless auditions, mesmerised by old Hollywood glamour. However, as the incurable romantics keep running into each other under the city's starry nights and plum-hued sunrises, a magical, old-school tap-dance romance timidly begins. Such is the power of effervescent love: it inspires people to have long walks while gazing at the colourful Milky Way of the city's flickering lights. But life is as exciting as it is challenging. After all, no one knows what tomorrow holds. And as the young idealists veer from their dreams, questions arise. Does true love exist only in movies? Do dreams come true in real life?
"""

In [ ]:
get_similarity_from_plot(user_input = text2, embeddings = embeddings, num_of_similars = 3, print_plot = False)

I recommend, ['Open Tee Bioscope', 'Seven Footprints to Satan', 'In Search of Gregory']


In [ ]:
text3 = """
Jim and his fiancee Eve, a young society couple, are kidnapped on the eve of Jim's departure for Africa and brought to a mansion that is home to a strange and glamorous cult with a hooded leader called "Satan." Jim is put through a number of strange adventures in the old house and tries to maintain his courage. During the course of the film, Jim encounters an old witch, a dwarf, a gorilla and a strange shaggy creature called "The Spider". In the end, he is confronted by Satan himself who puts him to a final test.

It is revealed to be a hoax played on Jim by his uncle Joe, Eve, and his uncle's employees to convince Jim to forget his adventure plans, stay at home, work for his uncle, and settle down with Eve.
"""
get_similarity_from_plot(
    user_input=text3, embeddings=embeddings, num_of_similars=3, print_plot=False)


I recommend, ['Seven Footprints to Satan', 'The Nun and the Bandit', 'Wu Kong']
